# Implementing an untyped Lambda Calculus


To introduce `metadsl` we will show how to create a working version of the untyped lambda calculus.

This will let us create the some basic combinators, verify that they before as expected under equiavelency.

Then we can also create an implementation of arithmetic on natural numbers, using church encoding and show how to convert
between these and Python's integers.


## Terms
The lambda calculus might sound abstract and odd, but if you know how to write functions in Python you already have mastered it!
The idea is a very simple language that only has functions that take in one argument.
Nothing, else, which seems a bit silly maybe, and it is not very useful by itself, but when it is integrated into a system with other data
types, it becomes a useful so that we can represent functions.


First, let's create a concept of a "variable" that can optionally have a name. It should only be equal to itself, so that we don't have to worry about shadowing variable names:

In [1]:
import dataclasses
import typing


@dataclasses.dataclass(eq=False)
class Variable:
    name: typing.Optional[str] = None
    
    
    def __str__(self):
        return self.name or str(id(self))
        
    def __repr__(self):
        return f"Variable({str(self)})"
        
x1 = Variable("x")
x2 = Variable("x")

assert x1 == x1
assert x1 != x2
assert hash(x1) != hash(x2)

Now, let's build our one and only key abstraction, a lambda term.
You should be able to call it and also create it by having a body and a variable. 

In [2]:
import metadsl


class LambdaTerm(metadsl.Instance):
    @metadsl.call(lambda self, arg: LambdaTerm)
    def __call__(self, arg: "LambdaTerm") -> "LambdaTerm":
        ...

@metadsl.call(lambda variable, body: LambdaTerm)
def abstraction(variable: Variable, body: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda variable: LambdaTerm)
def from_variable(variable: Variable) -> LambdaTerm:
    ...

def create_abstraction(fn: typing.Callable[[LambdaTerm], LambdaTerm]) -> LambdaTerm:
    arg = Variable()
    return abstraction(arg, fn(from_variable(arg)))

We can construct the identity function

In [3]:
@create_abstraction
def identity(x):
    return x

In [4]:
identity

LambdaTerm(_call=Call(abstraction, (Variable(4613453808), LambdaTerm(_call=Call(from_variable, (Variable(4613453808),))))))

and then call it on a term:

In [5]:
y_term = from_variable(Variable("y"))
called_identity = identity(y_term)
called_identity

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4613453808), LambdaTerm(_call=Call(from_variable, (Variable(4613453808),)))))), LambdaTerm(_call=Call(from_variable, (Variable(y),))))))

## Pretty Printing

This is a bit of a verbose way to print these.

Let's create a custom function that walks the tree and converts them to a string:

In [6]:
pretty_print_rules = metadsl.RulesRepeatFold()


@metadsl.call(lambda s: LambdaTerm)
def string_term(s: str) -> LambdaTerm:
    ...


@pretty_print_rules.append
@metadsl.rule(None)
def pp_variable(variable: Variable):
    return (
        from_variable(variable),
        lambda: string_term(str(variable))
    )

@pretty_print_rules.append
@metadsl.rule(None, None)
def pp_abstraction(variable: Variable, body: str):
    return (
        abstraction(variable, string_term(body)),
        lambda: string_term(f"(λ{variable}.{body})")
    )


@pretty_print_rules.append
@metadsl.rule(None, None)
def pp_application(fn: str, arg: str):
    return (
        string_term(fn)(string_term(arg)),
        lambda: string_term(f"({fn} {arg})"),
    )

@metadsl.call(lambda t: metadsl.RuleApplier(pretty_print_rules))
def unbox_str(t: LambdaTerm) -> str:
    ...


@pretty_print_rules.append
@metadsl.pure_rule(None)
def pp_unbox(s: str):
    return unbox_str(string_term(s)), s

In [7]:
unbox_str(called_identity)

'((λ4613453808.4613453808) y)'

## Beta Reduction

To make this useful, we need to define some semantics on these.

What's a useful thing to do with the lambda calculus?

Well it would be nice to actually be able to "compute" these values by replacing all instances of a variable with it's argument.

So let's replace all instances of calling a variable by replacing the arg inside of it:

In [14]:
beta_reduce_rules = metadsl.RulesRepeatFold()
beta_reduce_applier = metadsl.RuleApplier(beta_reduce_rules)

@metadsl.call(lambda t: beta_reduce_applier)
def beta_reduce(t: LambdaTerm) -> LambdaTerm:
    ...

@beta_reduce_rules.append
@metadsl.pure_rule(LambdaTerm)
def _remove_beta_reduce(t: LambdaTerm):
    return beta_reduce(t), t
    
@beta_reduce_rules.append
@metadsl.rule(None, LambdaTerm, LambdaTerm)
def _beta_reduce(var: Variable, body: LambdaTerm, arg: LambdaTerm):
    def replacement(var=var, body=body, arg=arg):
        # replaces all instances of var with arg
        replace_rules = metadsl.RulesRepeatFold(
            metadsl.pure_rule()(lambda: (from_variable(var), arg))
        )

        return metadsl.RuleApplier(replace_rules)(body)

    return (
        abstraction(var, body)(arg),
        replacement
    )
beta_reduce_applier(called_identity)

LambdaTerm(_call=Call(from_variable, (Variable(y),)))

Now let's combine these two, to execute them in sequence. First we want to beta reduce, until there is no more to beta reduce, then
we wanna turn that into a pretty print representation:

In [17]:
def execute(expr: LambdaTerm) -> str:
    return unbox_str(beta_reduce(expr))

In [18]:
print(execute(called_identity))

y


## Alpha Conversion

This looks allright, but before we do anything serious, let's deal with the annoying issue of these variable names. They aren't much fun to read! Let's fix this by implementing [α-conversion](https://en.wikipedia.org/wiki/Lambda_calculus#%CE%B1-conversion). The basic idea is that given some lambda expression, we rename all the variables to make them standard. Let's create a bunch of global variables, and then create a function that will transform an expression into a normal form using them:

In [19]:
GLOBAL_VARS = [Variable(chr(i)) for i in range(ord('a'), ord('a') + 100)]
CURRENT_GLOBAL_VAR = 0

In [20]:
GLOBAL_VARS[0]

Variable(a)

The idea is that we will map each variable we find in the expression to one of the global variables:

In [22]:
alpha_convert_rules = metadsl.RulesRepeatFold()

def execute_alpha_convert(x):
    global CURRENT_GLOBAL_VAR
    CURRENT_GLOBAL_VAR = 0
    return metadsl.RuleApplier(alpha_convert_rules)(x)

@metadsl.call(lambda t: execute_alpha_convert)
def alpha_convert(t: LambdaTerm) -> LambdaTerm:
    ...

@alpha_convert_rules.append
@metadsl.pure_rule(LambdaTerm)
def _remove_alpha_convert(t: LambdaTerm):
    return alpha_convert(t), t


@alpha_convert_rules.append
@metadsl.rule(None, LambdaTerm)
def _alpha_convert(var: Variable, body: LambdaTerm):
    def replace(var=var, body=body):
        if var in GLOBAL_VARS: 
            return None
        global CURRENT_GLOBAL_VAR

        new_var = GLOBAL_VARS[CURRENT_GLOBAL_VAR]
        CURRENT_GLOBAL_VAR += 1

        replace_rules = metadsl.RulesRepeatFold(
            metadsl.pure_rule()(lambda: (var, new_var))
        )
 
        res = abstraction(new_var, metadsl.RuleApplier(replace_rules)(body))
        
        # We have already replaced this if we got the same
        if res == abstraction(var, body):
            return None
        return res
    
    return (
        abstraction(var, body),
        replace
    )

In [23]:
identity

LambdaTerm(_call=Call(abstraction, (Variable(4613453808), LambdaTerm(_call=Call(from_variable, (Variable(4613453808),))))))

In [24]:
alpha_convert(identity)

LambdaTerm(_call=Call(abstraction, (Variable(a), LambdaTerm(_call=Call(from_variable, (Variable(a),))))))

Now let's add this to our execution step, so that we print before and after beta reducing:

In [73]:
core_replacements = metadsl.RulesRepeatSequence(beta_reduce_rules)
core_applier = metadsl.RuleApplier(core_replacements)

def execute(expr: LambdaTerm) -> str:
    expr = alpha_convert(expr)
    return f"{unbox_str(expr)} ⇒ {unbox_str(alpha_convert(core_applier(expr)))}"


In [74]:
execute(identity(identity))

'((λa.a) (λb.b)) ⇒ (λb.b)'

Now we are finally getting somewhere!

In [75]:
execute(identity(identity)(from_variable(Variable("Input"))))

'(((λa.a) (λb.b)) Input) ⇒ Input'

Let's actually just set it to always pretty print this representation:

In [76]:
text_formatter = get_ipython().display_formatter.formatters['text/html']
text_formatter.for_type(LambdaTerm, lambda e, *args: execute(e))

<function __main__.<lambda>(e, *args)>

In [77]:
identity

LambdaTerm(_call=Call(abstraction, (Variable(4613453808), LambdaTerm(_call=Call(from_variable, (Variable(4613453808),))))))

In [78]:
identity(identity)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4613453808), LambdaTerm(_call=Call(from_variable, (Variable(4613453808),)))))), LambdaTerm(_call=Call(abstraction, (Variable(4613453808), LambdaTerm(_call=Call(from_variable, (Variable(4613453808),)))))))))

## Church Numerals

Now that we have a working system, we can start to implement some more useful programming on top of it.

One thing we can do is [represent natural numbers](https://en.wikipedia.org/wiki/Church_encoding#Church_numerals) with lambda expression and do math on them.

### Python Ints
First, let's add the ability to embed Python's ints inside a function:

In [79]:
@metadsl.call(lambda i: LambdaTerm)
def int_term(i: int) -> LambdaTerm:
    ...

In [80]:
int_term(123)

LambdaTerm(_call=Call(int_term, (123,)))

Oops! We don't know how to turn a int represenation of a lambda term into a string, lets fix that:

In [81]:
@pretty_print_rules.append
@metadsl.rule(None)
def int_to_str(i: int):
    return int_term(i), lambda: string_term(str(i))

In [82]:
identity(int_term(123))

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4613453808), LambdaTerm(_call=Call(from_variable, (Variable(4613453808),)))))), LambdaTerm(_call=Call(int_term, (123,))))))

### Church Numerals

How do we create a church numeral? Well we have a function that calls itself $n$ times, to represent $n$:

In [83]:
def church_numeral(n: int) -> LambdaTerm:
    @create_abstraction
    def outer(f):
        
        @create_abstraction
        def inner(x):
            for i in range(n):
                x = f(x)
            return x
        return inner
    return outer

In [84]:
church_numeral(0)

LambdaTerm(_call=Call(abstraction, (Variable(4621969504), LambdaTerm(_call=Call(abstraction, (Variable(4621970848), LambdaTerm(_call=Call(from_variable, (Variable(4621970848),)))))))))

In [85]:
x = church_numeral(1)
x

LambdaTerm(_call=Call(abstraction, (Variable(4621898136), LambdaTerm(_call=Call(abstraction, (Variable(4621943192), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621898136),))), LambdaTerm(_call=Call(from_variable, (Variable(4621943192),))))))))))))

In [86]:
church_numeral(2)

LambdaTerm(_call=Call(abstraction, (Variable(4620388728), LambdaTerm(_call=Call(abstraction, (Variable(4620390352), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620388728),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620388728),))), LambdaTerm(_call=Call(from_variable, (Variable(4620390352),)))))))))))))))

In [87]:
church_numeral(3)

LambdaTerm(_call=Call(abstraction, (Variable(4623729552), LambdaTerm(_call=Call(abstraction, (Variable(4620276568), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4623729552),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4623729552),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4623729552),))), LambdaTerm(_call=Call(from_variable, (Variable(4620276568),))))))))))))))))))

Now let's add some replacements that from python integers to church numerals:

In [88]:
to_church_rules = metadsl.RulesRepeatFold()
apply_to_church = metadsl.RuleApplier(to_church_rules)

@to_church_rules.append
@metadsl.rule(None)
def int_to_abstraction(i: int):
    return int_term(i), lambda: church_numeral(i)

In [89]:
apply_to_church(int_term(10))

LambdaTerm(_call=Call(abstraction, (Variable(4621969952), LambdaTerm(_call=Call(abstraction, (Variable(4621875856), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621969952),))), LambdaTerm(_call=Call(from_variable, (Variable(4621875856),)))))))))))))))))))))))))))))))))))))))

### Math

Add finally we can define math on these church numerals:

In [90]:
@metadsl.call(lambda x, y: LambdaTerm)
def add(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda x: LambdaTerm)
def succ(x: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda x, y: LambdaTerm)
def mult(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda x, y: LambdaTerm)
def exp(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

#### on Python Integers
First, we can define how these functions behave on Python integers:

In [91]:
int_math_rules = metadsl.RulesRepeatFold()
apply_int_math = metadsl.RuleApplier(int_math_rules)


@int_math_rules.append
@metadsl.rule(None, None)
def _add_ints(x: int, y: int):
    return add(int_term(x), int_term(y)), lambda: int_term(x + y)

@int_math_rules.append
@metadsl.rule(None)
def _succ_int(x: int):
    return succ(int_term(x)), lambda: int_term(x + 1)

@int_math_rules.append
@metadsl.rule(None, None)
def _mult_int(x: int, y: int):
    return mult(int_term(x), int_term(y)), lambda: int_term(x * y)


@int_math_rules.append
@metadsl.rule(None, None)
def _exp_int(x: int, y: int):
    return exp(int_term(x), int_term(y)), lambda: int_term(x ** y)

Now we can add these to our core replacements that we defined above and our executed during printing:

In [92]:
core_replacements.append(int_math_rules)

RulesRepeatFold(rules=(InferredMatchRule(arg_types=(None, None), match_function=<function _add_ints at 0x1136712f0>, match_rule=MatchRule(template=RecursiveCall(add, (RecursiveCall(int_term, (Wildcard(),)), RecursiveCall(int_term, (Wildcard(),)))), match=<bound method InferredMatchRule._match of ...>), arg_wildcards=(Wildcard(), Wildcard())), InferredMatchRule(arg_types=(None,), match_function=<function _succ_int at 0x113671378>, match_rule=MatchRule(template=RecursiveCall(succ, (RecursiveCall(int_term, (Wildcard(),)),)), match=<bound method InferredMatchRule._match of ...>), arg_wildcards=(Wildcard(),)), InferredMatchRule(arg_types=(None, None), match_function=<function _mult_int at 0x113671400>, match_rule=MatchRule(template=RecursiveCall(mult, (RecursiveCall(int_term, (Wildcard(),)), RecursiveCall(int_term, (Wildcard(),)))), match=<bound method InferredMatchRule._match of ...>), arg_wildcards=(Wildcard(), Wildcard())), InferredMatchRule(arg_types=(None, None), match_function=<functi

In [93]:
add(int_term(1), int_term(2))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(int_term, (1,))), LambdaTerm(_call=Call(int_term, (2,))))))

Oh, we forgot to define the pretty printing version of these operations. Let's do that now as well:

In [94]:
@pretty_print_rules.append
@metadsl.rule(None, None)
def _add_str(x: str, y: str):
    return add(string_term(x), string_term(y)), lambda: string_term(f"{x} + {y}")

@pretty_print_rules.append
@metadsl.rule(None)
def _succ_str(x: str):
    return succ(string_term(x)), lambda: string_term(f"succ({x})")

@pretty_print_rules.append
@metadsl.rule(None, None)
def _mult_str(x: str, y: str):
    return mult(string_term(x), string_term(y)), lambda: string_term(f"{x} * {y}")

@pretty_print_rules.append
@metadsl.rule(None, None)
def _exp_str(x: str, y: str):
    return exp(string_term(x), string_term(y)), lambda: string_term(f"{x}^{y}")


In [95]:
add(int_term(1), int_term(2))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(int_term, (1,))), LambdaTerm(_call=Call(int_term, (2,))))))

Sweet!

#### on Church numerals
But what if we turn them into church numerals first?

In [96]:
apply_to_church(add(int_term(1), int_term(2)))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(abstraction, (Variable(4623825944), LambdaTerm(_call=Call(abstraction, (Variable(4623796208), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4623825944),))), LambdaTerm(_call=Call(from_variable, (Variable(4623796208),)))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4623794696), LambdaTerm(_call=Call(abstraction, (Variable(4623795760), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4623794696),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4623794696),))), LambdaTerm(_call=Call(from_variable, (Variable(4623795760),))))))))))))))))))

Well we don't know how to add those. First, let's define the operations as abstractions themselves (following wikipedia here on the naming):

In [97]:
@create_abstraction
def add_abstraction(m):
    @create_abstraction
    def inner(n):
        @create_abstraction
        def inner(f):
            @create_abstraction
            def inner(x):
                return m(f)(n(f)(x))
            return inner
        return inner
    return inner

@create_abstraction
def succ_abstraction(n):
    @create_abstraction
    def inner(f):
        @create_abstraction
        def inner(x):
            return f(n(f)(x))
        return inner
    return inner

@create_abstraction
def mult_abstraction(m):
    @create_abstraction
    def inner(n):
        @create_abstraction
        def inner(f):
            return m(n(f))
        return inner
    return inner

@create_abstraction
def exp_abstraction(m):
    @create_abstraction
    def inner(n):
        return n(m)
    return inner


Is this right? Well let's try it out!

In [98]:
zero = apply_to_church(int_term(0))
zero

LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),)))))))))

In [99]:
add_abstraction(zero)(zero)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4620328184), LambdaTerm(_call=Call(abstraction, (Variable(4620326056), LambdaTerm(_call=Call(abstraction, (Variable(4620325944), LambdaTerm(_call=Call(abstraction, (Variable(4620325160), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328184),))), LambdaTerm(_call=Call(from_variable, (Variable(4620325944),)))))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620326056),))), LambdaTerm(_call=Call(from_variable, (Variable(4620325944),)))))), LambdaTerm(_call=Call(from_variable, (Variable(4620325160),))))))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),)))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),))))))))))))

In [100]:
zero

LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),)))))))))

Sweet! Those look same, that's good. 

And we should find that adding one is the same as the succesor functions...

In [101]:
one = apply_to_church(int_term(1))

In [102]:
add_abstraction(one)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4620328184), LambdaTerm(_call=Call(abstraction, (Variable(4620326056), LambdaTerm(_call=Call(abstraction, (Variable(4620325944), LambdaTerm(_call=Call(abstraction, (Variable(4620325160), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328184),))), LambdaTerm(_call=Call(from_variable, (Variable(4620325944),)))))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620326056),))), LambdaTerm(_call=Call(from_variable, (Variable(4620325944),)))))), LambdaTerm(_call=Call(from_variable, (Variable(4620325160),))))))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4620328576), LambdaTerm(_call=Call(abstraction, (Variable(4620326168), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328576),))), LambdaTerm(_call=Call(from_variable, (Variable(4620326168),)))))))))))))))

In [103]:
succ_abstraction

LambdaTerm(_call=Call(abstraction, (Variable(4620327008), LambdaTerm(_call=Call(abstraction, (Variable(4620279200), LambdaTerm(_call=Call(abstraction, (Variable(4623795816), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620279200),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620327008),))), LambdaTerm(_call=Call(from_variable, (Variable(4620279200),)))))), LambdaTerm(_call=Call(from_variable, (Variable(4623795816),))))))))))))))))))

Those look the same! Nice!

OK What about multiplying by zero, that should always return zero...

In [104]:
mult_abstraction(zero)(from_variable(Variable("doesnt matter")))

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4620194368), LambdaTerm(_call=Call(abstraction, (Variable(4620194536), LambdaTerm(_call=Call(abstraction, (Variable(4623745992), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620194368),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620194536),))), LambdaTerm(_call=Call(from_variable, (Variable(4623745992),)))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),)))))))))))), LambdaTerm(_call=Call(from_variable, (Variable(doesnt matter),))))))

Yep!

It's kind cool how exponentiation is now so simple... Let's try raising something to the 1st power, this should give us back itself:

In [105]:
exp_abstraction

LambdaTerm(_call=Call(abstraction, (Variable(4623747056), LambdaTerm(_call=Call(abstraction, (Variable(4621817952), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621817952),))), LambdaTerm(_call=Call(from_variable, (Variable(4623747056),))))))))))))

In [106]:
exp_abstraction(from_variable(Variable("doesnt matter")))(one)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4623747056), LambdaTerm(_call=Call(abstraction, (Variable(4621817952), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621817952),))), LambdaTerm(_call=Call(from_variable, (Variable(4623747056),)))))))))))), LambdaTerm(_call=Call(from_variable, (Variable(doesnt matter),)))))), LambdaTerm(_call=Call(abstraction, (Variable(4620328576), LambdaTerm(_call=Call(abstraction, (Variable(4620326168), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328576),))), LambdaTerm(_call=Call(from_variable, (Variable(4620326168),)))))))))))))))

This is actually to the input, despite it different looking output. If we had implemented [Eta-conversion](https://en.wikipedia.org/wiki/Lambda_calculus#%CE%B7-conversion) this would reduce to "doesn't matter".

Now we can map these to the operations we defined above:

In [107]:
church_math_rules = metadsl.RulesRepeatFold()
core_replacements.append(church_math_rules)

@church_math_rules.append
@metadsl.pure_rule(None, None)
def _add(x: LambdaTerm, y: LambdaTerm):
    return add(x, y), add_abstraction(x)(y)

@church_math_rules.append
@metadsl.pure_rule(None)
def _succ(x: LambdaTerm):
    return succ(x), succ(x)

@church_math_rules.append
@metadsl.pure_rule(None, None)
def _mult(x: LambdaTerm, y: LambdaTerm):
    return mult(x, y), mult_abstraction(x)(y)

@church_math_rules.append
@metadsl.pure_rule(None, None)
def _mult(x: LambdaTerm, y: LambdaTerm):
    return mult(x, y), mult_abstraction(x)(y)


In [108]:
add(zero, one)

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4620328576), LambdaTerm(_call=Call(abstraction, (Variable(4620326168), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328576),))), LambdaTerm(_call=Call(from_variable, (Variable(4620326168),)))))))))))))))

In [109]:
one

LambdaTerm(_call=Call(abstraction, (Variable(4620328576), LambdaTerm(_call=Call(abstraction, (Variable(4620326168), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328576),))), LambdaTerm(_call=Call(from_variable, (Variable(4620326168),))))))))))))

In [111]:
add(int_term(10), int_term(5))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(int_term, (10,))), LambdaTerm(_call=Call(int_term, (5,))))))

In [113]:
apply_to_church(add(int_term(10), int_term(5)))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(abstraction, (Variable(4621875184), LambdaTerm(_call=Call(abstraction, (Variable(4621874400), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621875184),))), LambdaTerm(_call=Call(from_variable, (Variable(4621874400),))))))))))))))))))))))))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4621877144), LambdaTerm(_call=Call(abstraction, (Variable(4621874904), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621877144),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621877144),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621877144),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621877144),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4621877144),))), LambdaTerm(_call=Call(from_variable, (Variable(4621874904),)))))))))))))))))))))))))))

So now we have developed al little lambda calculus calculator. We can define custom pipelines now, reausing all of this work, to apply the steps in any order we would like.

And if we have users, they can right functions with the functions, and not worry how they are compiled. For example, I could write a function like this:

In [114]:
def my_special_math(x, y, z):
    return mult(add(x, y), z)

And the function is totally seperate from what we call it with, or how it ends up being compiled:

In [115]:
my_special_math(int_term(10), int_term(10), int_term(10))

LambdaTerm(_call=Call(mult, (LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(int_term, (10,))), LambdaTerm(_call=Call(int_term, (10,)))))), LambdaTerm(_call=Call(int_term, (10,))))))

In [116]:
my_special_math(zero, one, one)

LambdaTerm(_call=Call(mult, (LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4620328576), LambdaTerm(_call=Call(abstraction, (Variable(4620326168), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328576),))), LambdaTerm(_call=Call(from_variable, (Variable(4620326168),))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4620328576), LambdaTerm(_call=Call(abstraction, (Variable(4620326168), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4620328576),))), LambdaTerm(_call=Call(from_variable, (Variable(4620326168),)))))))))))))))

In [118]:
apply_to_church(my_special_math(zero, int_term(10), int_term(10)))

LambdaTerm(_call=Call(mult, (LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(abstraction, (Variable(4613953408), LambdaTerm(_call=Call(abstraction, (Variable(4623689488), LambdaTerm(_call=Call(from_variable, (Variable(4623689488),))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4619274112), LambdaTerm(_call=Call(abstraction, (Variable(4619274000), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4619274112),))), LambdaTerm(_call=Call(from_variable, (Variable(4619274000),)))))))))))))))))))))))))))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4624030688), LambdaTerm(_call=Call(abstraction, (Variable(4624031192), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4624030688),))), LambdaTerm(_call=Call(from_variable, (Variable(4624031192),))))))))))))))))))))))))))))))))))))))))))

For another tutorial, it would be useful to show how this compares to a *typed* lambda calculus. That would let us deal with functions over multiple data types, so we know if operations like addition even make sense.